In [ ]:
#4) Cross join
'''
CREATE TABLE A( ID int);
INSERT INTO A(ID) VALUES (1,1,1,2,2,3,NULL);

CREATE TABLE B(ID int);
INSERT INTO B(ID) VALUES (1,2,3,NULL);


########## SOLUTION #########
1)
Select * from A INNER JOIN B ON A.ID = B.ID;
Number of Records: 6
ID
1 ,1, 2 ,2 ,3, 1

2)Select * from A LEFT JOIN B ON A.ID = B.ID;
Number of Records: 7
ID
1, 1, 2 ,2, 3, null ,1

3)Select * from A RIGHT JOIN B ON A.ID = B.ID;
Number of Records: 7
ID
1 ,1, 1, 2, 2, 3, null
'''

In [ ]:
#Give customer_names of the persons who have not bought 'C'
'''
      -- Customers table:
    -- +-------------+---------------+
    -- | customer_id | customer_name |
    -- +-------------+---------------+
    -- | 1           | Daniel        |
    -- | 2           | Diana         |
    -- | 3           | Elizabeth     |
    -- | 4           | Jhon          |
    -- +-------------+---------------+
    -- Orders table:
    -- +------------+--------------+---------------+
    -- | order_id   | customer_id  | product_name  |
    -- +------------+--------------+---------------+
    -- | 10         |     1        |     A         |
    -- | 20         |     1        |     B         |
    -- | 30         |     1        |     D         |
    -- | 40         |     1        |     C         |
    -- | 50         |     2        |     A         |
    -- | 60         |     3        |     A         |
    -- | 70         |     3        |     B         |
    -- | 80         |     3        |     D         |
    -- | 90         |     4        |     C         |
'''


### SOLUTION 1
'''
  select customer_id,customer_name from customer table where customer_id not in 
      (SELECT distinct customer_id where product_name = 'C') #1,2
'''

### SOLTION 2
'''
  SELECT 
      customer_name
  FROM Customer c 
  LEFT JOIN Orders o
     ON c.customer_id = o.customer_id 
     AND o.product = 'C'
  WHERE o.order_id is NULL

#Explanation
  c.customer_id,c.customer_name,o.order_id,o.customer_id,o.product_name
   1,Daniel,40,1,C
   2,Diana,NULL,NULL,NULL
   3,Elizabeth,NULL,NULL,NULL
   4,Jhon,90,4,C
'''

### Solution 3 
'''
  select customer_name from customers c
      WHERE NOT EXISTS( SELECT 1 FROM ORDERS as O 
                        WHERE c.customer_id = o.customer_id 
                              and o.product_name = 'C' 
                      )

# STEP 1 => INNER QUERY :  ##Aisse kaun se customer hai , jinki id match kare or product bhi , 1,2
# STEP 2 => NOT EXISTS MEANS EXCLUDE those which we got in STEP 1
'''


In [ ]:
# Find Employees Hired in the Last 30 Days
'''
Q) Assuming there is a HireDate column in the Employees table, write a query to find all employees who were hired in the last 30 days.

SELECT 
    EmployeeID, 
    EmployeeName, 
    HireDate
FROM Employees
WHERE HireDate >= DATEADD(DAY, -30, GETDATE());
'''

In [ ]:
### Top N per Group**
'''
sales(product_id, salesperson_id, sales_amount)
Write a SQL query to find the top 3 sales amounts for each `salesperson_id`.

SELECT
         salesperson_id,
         product_id,
         sales_amount
     FROM (
         SELECT
             salesperson_id,
             product_id,
             sales_amount,
             ROW_NUMBER() OVER (PARTITION BY salesperson_id ORDER BY sales_amount DESC) AS rank
         FROM sales
     ) ranked_sales
     WHERE rank <= 3;
'''
### Approach 2 
## correlated subquery. Here’s how you can write the query:
'''
SELECT 
    s1.salesperson_id, 
    s1.sales_amount
FROM 
    Sales s1
WHERE 
    (
        SELECT COUNT(DISTINCT s2.sales_amount) 
        FROM Sales s2 
        WHERE s2.salesperson_id = s1.salesperson_id 
        AND s2.sales_amount > s1.sales_amount
    ) < 3
ORDER BY 
    s1.salesperson_id, s1.sales_amount DESC;


'''
####
'''
Explanation:
 * Correlated Subquery: The subquery (SELECT COUNT(DISTINCT s2.sales_amount) ...) counts how many distinct sales amounts are 
 greater than the current row's sales_amount for the same salesperson_id.

 * WHERE ... < 3: The query filters out rows where there are already 3 or more distinct sales amounts greater than the current one. 
 This ensures that only the top 3 sales amounts are returned.
 
 * ORDER BY s1.salesperson_id, s1.sales_amount DESC: Finally, the results are ordered by salesperson_id and then by sales amount in 
 descending order.

This approach avoids the use of window functions and directly filters the top 3 sales amounts using a correlated subquery.
'''

In [1]:
### 3. **Identifying Gaps in Sequences**
'''
orders(`order_id` (sequential integer), `order_date`, `customer_id`). 
Q) Write a SQL query to identify any missing `order_id` values in the sequence.
   
SELECT
         order_id + 1 AS missing_order_id
FROM orders o1
WHERE NOT EXISTS (
         SELECT 1 FROM orders o2 WHERE o2.order_id = o1.order_id + 1
)
AND o1.order_id < (SELECT MAX(order_id) FROM orders);
'''

'\n   - **Problem:** You have a table `orders` with columns \n   `order_id` (sequential integer), `order_date`, and `customer_id`. \n   Write a SQL query to identify any missing `order_id` values in the sequence.\n   \nSELECT\n         order_id + 1 AS missing_order_id\nFROM orders o1\nWHERE NOT EXISTS (\n         SELECT 1 FROM orders o2 WHERE o2.order_id = o1.order_id + 1\n)\nAND o1.order_id < (SELECT MAX(order_id) FROM orders);\n'

In [ ]:
### 4. **Finding Duplicates**
'''
employees(employee_id,name, email,hire_date)
Q) write a query to find all employees who have the same `email` but different `employee_id`.

SELECT
  email,
         COUNT(*) AS count
FROM employees
GROUP BY email
HAVING COUNT(*) > 1;
'''
# OR 
'''
 SELECT 
     email,
     COUNT(distinct employee_id)
 FROM employees 
 GROUP BY email HAVING COUNT(distinct employee_id) > 1
'''

'\n   - **Problem:** Given a table `employees` with columns `employee_id`, `name`, `email`, and `hire_date`, write a query to find all employees who have the same `email` but different `employee_id`.\n   - **Expected Solution:**\n\nSELECT\n         email,\n         COUNT(*) AS count\n     FROM employees\n     GROUP BY email\n     HAVING COUNT(*) > 1;\n'

In [1]:
### 5. **Complex Joins and Aggregation**
''' 
1. customers (customer_id,name,city)  
2. orders(order_id, customer_id, order_date, and amount)

Q) Write a query to find the total order amount for each customer, including customers who have not placed any orders.

SELECT
  c.customer_id,
  c.name,
  COALESCE(SUM(o.amount), 0) AS total_order_amount  

FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.name;

'''

'\n- **Problem:** You have two tables: \n   customers (customer_id,name,city) and \n   orders(order_id, `customer_id`, `order_date`, and `amount`). \n   Write a query to find the total order amount for each customer, including customers who have not placed any orders.\n\nSELECT\n         c.customer_id,\n         c.name,\n         COALESCE(SUM(o.amount), 0) AS total_order_amount\n     FROM customers c\n     LEFT JOIN orders o ON c.customer_id = o.customer_id\n     GROUP BY c.customer_id, c.name;\n\n'

In [ ]:
#### Large Problem
### **Problem: Inventory Management System**
'''
You are working with a database that handles inventory for an e-commerce company. The company stores information about its 
products, suppliers, and stock levels in the following tables:

1. **Products Table**
   - `ProductID`: Unique identifier for each product.
   - `ProductName`: Name of the product.
   - `Category`: Category to which the product belongs.

2. **Suppliers Table**
   - `SupplierID`: Unique identifier for each supplier.
   - `SupplierName`: Name of the supplier.
   - `Country`: Country where the supplier is located.

3. **Stock Table**
   - `ProductID`: Unique identifier for each product (foreign key from Products table).
   - `SupplierID`: Unique identifier for each supplier (foreign key from Suppliers table).
   - `Quantity`: Quantity of the product available in stock.

### **Question:**
You have been asked to provide insights into the current inventory situation. Specifically, you need to write SQL queries to answer 
the following questions:

1. **Top Suppliers by Stock Quantity:**
   Write a query to find the top 5 suppliers who have supplied the most quantity of products across all categories.
   select
      s.Supplier_name,
      s.supplier_id
      count(Quantity) as total_qunaityt 

   FROM Supplier s JOIN Stock st 
      ON s.supplier_id = st.supplier_id 
   GROUP BY st.supplier_id 
   ORDER BY total_qunaityt DESC LIMIT 5


2. **Out-of-Stock Products:**
   Write a query to identify all products that are currently out of stock.

3. **Category-Wise Stock Summary:**
   Write a query to summarize the total stock quantity for each product category.

4. **Suppliers with Low Stock Products:**
   Write a query to identify suppliers who are responsible for products that are low in stock (i.e., quantity less than 10). 

### Expected Output:
- Top Suppliers by Stock Quantity:** A list of supplier names along with the total quantity supplied, ordered by quantity in descending order.
- Out-of-Stock Products: A list of product names that have a stock quantity of zero.
- Category-Wise Stock Summary: A summary table showing each category and the total stock quantity available for that category.
- Suppliers with Low Stock Products: A list of supplier names who have at least one product with stock less than 10, along with the product name and current stock quantity.

### **Example Data:**
- Products Table:
  | ProductID | ProductName | Category  |
  |-----------|-------------|-----------|
  | 1         | Widget A    | Gadgets   |
  | 2         | Widget B    | Gadgets   |
  | 3         | Gadget C    | Gadgets   |
  | 4         | Tool D      | Tools     |
  | 5         | Tool E      | Tools     |

- Suppliers Table:
  | SupplierID | SupplierName | Country  |
  |------------|--------------|----------|
  | 1          | Supplier X   | USA      |
  | 2          | Supplier Y   | China    |
  | 3          | Supplier Z   | India    |

- Stock Table:
  | ProductID | SupplierID | Quantity |
  |-----------|------------|----------|
  | 1         | 1          | 50       |
  | 2         | 2          | 30       |
  | 3         | 3          | 0        |
  | 4         | 1          | 20       |
  | 5         | 2          | 5        |

This problem will test your ability to write SQL queries that aggregate data, join tables, and filter results based on specific conditions.
'''